# This notebook performs sentiment analysis on financial news headlines from Finviz for specific stock symbols. Here is a detailed summary of each step:


The first cell installs the necessary Python libraries: requests, beautifulsoup4, pandas, and vaderSentiment.
Importing Libraries:

The second cell imports the libraries: requests for making HTTP requests, BeautifulSoup from bs4 for web scraping, pandas for data manipulation, and SentimentIntensityAnalyzer from vaderSentiment for sentiment analysis.
Scraping Finviz News Headlines:

The third cell defines a function to scrape news headlines for a given stock symbol from the Finviz website.
Sentiment Analysis Function:

The fourth cell contains a function that performs sentiment analysis on the scraped headlines using VADER, a sentiment analysis tool that measures the polarity of text (positive, negative, neutral).
Main Execution Block:

The fifth cell is the main execution block. It lists stock symbols (e.g., ONCO, CNEY, TNXP, APLD, KTTA) and scrapes headlines for each symbol, analyzing their sentiment and storing the results.

In [1]:
!pip install requests beautifulsoup4 pandas vaderSentiment


  Obtaining dependency information for vaderSentiment from https://files.pythonhosted.org/packages/76/fc/310e16254683c1ed35eeb97386986d6c00bc29df17ce280aed64d55537e9/vaderSentiment-3.3.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/126.0 kB ? eta -:--:--
   ---------------------------------------- 126.0/126.0 kB 3.7 MB/s eta 0:00:00


## Installs the libraries needed for web scraping, data manipulation, and sentiment analysis.
requests: For sending HTTP requests to fetch web content.
beautifulsoup4: For parsing and scraping HTML content.
pandas: For handling data structures, such as data frames.
vaderSentiment: For performing sentiment analysis on text data.

In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## Defines a function to scrape financial news headlines for a specified stock from Finviz.
Uses the stock symbol to create the URL for the stock's news page.
Sends a GET request to the URL to fetch the page content.
Uses BeautifulSoup to parse the HTML and find the news table containing the headlines.
Extracts and returns a list of headlines from the table.

In [9]:

# Function to scrape headlines from Finviz for a specific stock
def get_finviz_news(stock_symbol):
    url = f"https://finviz.com/quote.ashx?t={stock_symbol}&p=d"  # Using the provided URL format
    
    # Add headers to mimic a real browser request
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract news table
    news_table = soup.find('table', class_='fullview-news-outer')
    
    # Handle the case where the table is not found
    if news_table is None:
        print(f"No news found for {stock_symbol}")
        return []
    
    # List to hold news
    news = []
    
    # Iterate through table rows
    for row in news_table.findAll('tr'):
        news_time = row.td.text.strip()  # Extract date and time
        headline = row.a.text.strip()    # Extract headline
        news.append({"Date": news_time, "Headline": headline})
    
    return news



## Defines a function that takes a list of headlines and performs sentiment analysis on each one.
Creates an instance of SentimentIntensityAnalyzer.
Loops through each headline and calculates the sentiment score, which includes positive, negative, neutral, and compound scores.
Prints the headline along with its sentiment scores.

In [10]:
# Perform sentiment analysis on the headlines
def analyze_sentiment(headlines):
    analyzer = SentimentIntensityAnalyzer()
    
    # Analyze sentiment for each headline
    for headline in headlines:
        sentiment_score = analyzer.polarity_scores(headline['Headline'])['compound']
        headline['Sentiment'] = sentiment_score
    
    return headlines



## Demonstrates how to use the functions to get headlines and analyze sentiment.
Defines a list of stock symbols to analyze.
Scrapes headlines for each stock symbol and stores them in a list.
Creates a pandas DataFrame to organize the headlines.
Calls the analyze_sentiment function to evaluate the sentiment of each headline.

In [11]:
# Example usage
if __name__ == "__main__":
    symbols = ['ONCO', 'CNEY', 'TNXP', 'APLD', 'KTTA']  # List of stock symbols
    
    # List to hold all headlines and their sentiment
    all_headlines = []
    
    for symbol in symbols:
        # Scrape news for each symbol
        headlines = get_finviz_news(symbol)
        # Perform sentiment analysis
        if headlines:
            headlines_with_sentiment = analyze_sentiment(headlines)
            # Add stock symbol to the data
            for headline in headlines_with_sentiment:
                headline['Stock Symbol'] = symbol
            all_headlines.extend(headlines_with_sentiment)
    
    # Create a DataFrame from the list of headlines with sentiment
    headlines_df = pd.DataFrame(all_headlines)
    
    # Display the DataFrame
    print(headlines_df[['Stock Symbol', 'Date', 'Headline', 'Sentiment']])

  

    Stock Symbol               Date  \
0           ONCO  Sep-20-24 11:45AM   
1           ONCO  Jul-15-24 06:00AM   
2           ONCO  Jul-11-24 08:17AM   
3           ONCO  May-30-24 07:47AM   
4           ONCO  May-21-24 09:52AM   
..           ...                ...   
421         KTTA            07:30AM   
422         KTTA  Nov-11-21 08:00AM   
423         KTTA  Nov-09-21 08:00AM   
424         KTTA  Oct-27-21 11:40AM   
425         KTTA            07:45AM   

                                              Headline  Sentiment  
0    Onconetix, Inc. Announces 1-for-40 Reverse Sto...     0.0000  
1    Onconetix Announces Closing of Warrant Exercis...    -0.4767  
2    Onconetix Announces Exercise of Warrants for $...    -0.4767  
3    Today's Biggest Pre-Market Stock Movers: 10 To...    -0.3818  
4    ONCO Stock Earnings: Onconetix Reported Result...     0.0000  
..                                                 ...        ...  
421  EXCLUSIVE: Pasithea Clinics Launches In-Home I... 

## Saves the DataFrame containing the stock symbols and headlines to a CSV file.

In [12]:
  # Optionally, save to CSV for further analysis
headlines_df.to_csv('finviz_news_sentiment.csv', index=False)
